## Youtude 랭킹 1000

In [97]:
import requests, time       # requests 인터넷으로 자료를 받아서 가지고 올 때 사용 
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote
from selenium import webdriver

- Step 1, 2

In [98]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [99]:
trs = soup.select('tr.aos-init')
len(trs)

0

In [100]:
driver = webdriver.Chrome() 
driver.get(url)

In [101]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

- Step 3

In [102]:
tr = trs[0]
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [103]:
category = tr.select_one('.category').get_text().strip()[1:-1]       # '[음악/댄스/가수]'
category


'음악/댄스/가수'

In [104]:
channel = tr.select_one('.subject > h1 > a').get_text().strip()     # h1테그와 a테그가 여러개임으로 거꾸로 올라가서 subject에 h1, a 라는걸 지정해줌
channel

'BLACKPINK'

In [105]:
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
subscriber

'9130만'

In [106]:
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
view,video

('332억0683만', '567개')

In [107]:
# 숫자로 바꿔주는 함수 
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [108]:
convert('332억0683만')          # 유튜브 랭크에 뷰카운트를 바꿔준 것이 아니라 어느 문자를 넣어도 숫자로 바꿔줌

33206830000

- Step 4

In [109]:
data = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())       # 아까 조회수 함수를 숫자로 바꿔주는 convert 를 만들었으니 붙여준다.
    view = convert(tr.select_one('.view_cnt').get_text().strip())                   # 동일
    video = convert(tr.select_one('.video_cnt').get_text().strip())                 # 동일
    data.append({'순위':rank, '채널명':channel, '카테고리':category,
                 '구독자수':subscriber, '조회수':view, '비디오수':video})
    

- Step 5. 에러 수정 

In [110]:
# 에러가 난 위치를 확인하는 방법 

try:
    for index, tr in enumerate(trs):
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())       # 아까 재생수 함수를 숫자로 바꿔주는 convert 를 만들었으니 붙여준다.
        view = convert(tr.select_one('.view_cnt').get_text().strip())                   # 동일
        video = convert(tr.select_one('.video_cnt').get_text().strip())                 # 동일
except:
    print(index + 1)        # 인덱스 순번으로 나옴

In [111]:
# 오류 해결 후 다시 조회

data = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())       # 아까 조회수 함수를 숫자로 바꿔주는 convert 를 만들었으니 붙여준다.
    view = convert(tr.select_one('.view_cnt').get_text().strip())                   # 동일
    video = convert(tr.select_one('.video_cnt').get_text().strip())                 # 동일
    data.append({'순위':rank, '채널명':channel, '카테고리':category,
                 '구독자수':subscriber, '조회수':view, '비디오수':video})

- Step 6

In [112]:

data = []
for page in range(1, 11):
    url = f'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page={page}'
    driver.get(url)
    # requests.get() => HTML 데이터만 가져오면 됨
    # selenium : HTML 데이터를 가져오고 브라우저에 화면에 표시해줘야함으로 바로 실행하면,
    #      반복문에서 뺑뺑이 돌기때문에 실행되는 시간이 없어 에러남으로, time.sleep으로 시간텀을줌
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('tr.aos-init')

    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())       # 아까 조회수 함수를 숫자로 바꿔주는 convert 를 만들었으니 붙여준다.
        view = convert(tr.select_one('.view_cnt').get_text().strip())                   # 동일
        video = convert(tr.select_one('.video_cnt').get_text().strip())                 # 동일
        data.append({'순위':rank, '채널명':channel, '카테고리':category,
                    '구독자수':subscriber, '조회수':view, '비디오수':video})
        
driver.close()

- Step 7

In [113]:
df = pd.DataFrame(data)
df.head()

,순위,채널명,카테고리,구독자수,조회수,비디오수
0,1,BLACKPINK,음악/댄스/가수,91300000,33206830000,567
1,2,BANGTANTV,음악/댄스/가수,76400000,21348920000,2330
2,3,HYBE LABELS,음악/댄스/가수,72700000,29979900000,1428
3,4,SMTOWN,음악/댄스/가수,32100000,27856680000,4227
4,5,JYP Entertainment,음악/댄스/가수,27800000,20510660000,1833


In [114]:
df.to_csv('data/유튜브랭킹_20230919.csv', index=False)
pd.read_csv('data/유튜브랭킹_20230919.csv').tail()

,순위,채널명,카테고리,구독자수,조회수,비디오수
995,997,성창경TV,미분류,770000,1196450000,18314
996,996,빅페이스 BIGFACE,미분류,770000,183120000,142
997,998,꾸삐KUPI,키즈/어린이,770000,570580000,986
998,1000,서은이야기 [V-log],키즈/어린이,770000,298990000,628
999,999,LDF TV by lottedutyfree,TV/방송,770000,656800000,1517
